# Setup code for implementing Redis Key Value DB into Docker compose

## Environment variables setup

In [1]:
"""REDIS_PORT=6379
REDIS_HOST=redis"""

redis_port=6379
redis_host="redis_parking_spots_status"




Modify docker compose .env so that it has access to redis port in env file

In [2]:
#append to docker-compose shared env neccessarry env variables so that the service could config properly
with open('../.env', 'a') as f:
    f.write(f"\n\nREDIS_PORT={redis_port}")

Create shared .env for Redis, MobileApp and ManagerApp

In [3]:
#Create shared env file for ManagerApp and timescaleDB config
shared_env_app = f'''
REDIS_PORT={redis_port}
REDIS_HOST={redis_host}
'''  

with open('.env.shared.ManagerApp.MobileApp', 'w') as f:
    f.write(shared_env_app)

## Redis Dockerfile setup

Dockerfile for Redis must be created

In [4]:
%%writefile Dockerfile
# Use the official Redis image from the Docker Hub
FROM redis:latest



# Expose the default Redis port
EXPOSE 6379


Writing Dockerfile


## Modify docker compose

In [11]:
"""version: '3'


services:
  redis:
    build: ./redis
    container_name: redis_container
    ports:
      - "${REDIS_PORT}:6379"
    environment:
      - REDIS_PORT=${REDIS_PORT}
      - REDIS_HOST=${REDIS_HOST}
    healthcheck:
      test: ["CMD", "redis-cli", "ping"]
      interval: 30s
      timeout: 10s
      retries: 3
      
  

"""



from ruamel.yaml import YAML

# Initialize YAML parser
yaml = YAML()
yaml.preserve_quotes = True  # Preserves quotes in the YAML file
yaml.indent(mapping = 2, sequence = 2, offset = 2)

#Setup file edit path 
docker_compose_path = '../docker-compose.yml'  

# Read the docker-compose.yml file
with open(docker_compose_path, 'r') as file:
    docker_compose = yaml.load(file)


redis_db = {
    'build' : './redis',
    'restart':'always',
    'ports' : [
        "${REDIS_PORT}:6379"
    ],
    'env_file' : [
        './redis/.env.shared.ManagerApp.MobileApp'
    ],
    'volumes' : [
        'redis_DB_parking_spot_status:/data'
    ],
    'networks' : [
        'app-network'
    ],
    'healthcheck' : {
      'test': ["CMD", "redis-cli","ping"], 
      'interval': '30s', 
      'timeout': '10s', 
      'retries':5, 
    }
}

if docker_compose['services']['manager_app']['depends_on'] is None:
    docker_compose['services']['manager_app']['depends_on']=dict()
docker_compose['services']['manager_app']['depends_on'].update({ 

    redis_host: { 
        'condition': 'service_healthy'
        },
})

if docker_compose['services']['mobile_app']['depends_on'] is None:
    docker_compose['services']['mobile_app']['depends_on']=dict()
docker_compose['services']['mobile_app']['depends_on'].update({ 

    redis_host: { 
        'condition': 'service_healthy'
        },
})


if docker_compose['services']['manager_app']['env_file'] is None:
    docker_compose['services']['manager_app']['env_file'] =[]
docker_compose['services']['manager_app']['env_file'] += ['./redis/.env.shared.ManagerApp.MobileApp']


if docker_compose['services']['mobile_app']['env_file'] is None:
    docker_compose['services']['mobile_app']['env_file'] =[]
docker_compose['services']['mobile_app']['env_file'] += ['./redis/.env.shared.ManagerApp.MobileApp']

# Add the service to the services section
docker_compose['services'][redis_host] = redis_db

docker_compose['volumes']['redis_DB_parking_spot_status'] = {}

# Write the updated configuration back to docker-compose.yml
with open(docker_compose_path, 'w') as file:
    yaml.dump(docker_compose, file)

print("\ndocker-compose.yml has been updated successfully.")



docker-compose.yml has been updated successfully.


# Redis testing




In [16]:
import redis

#!/bin/bash
# This script will initialize Redis with parking spot tracking data

# Define a parking spot with parking lot ID, parking spot ID, and availability
#redis-cli HMSET parking_spot:1:1 parking_lot_id 1 parking_spot_id 1 is_free 1
#redis-cli HMSET parking_spot:1:2 parking_lot_id 1 parking_spot_id 2 is_free 0
#redis-cli HMSET parking_spot:2:1 parking_lot_id 2 parking_spot_id 1 is_free 1
#redis-cli HMSET parking_spot:2:2 parking_lot_id 2 parking_spot_id 2 is_free 1

# Additional parking spots can be added similarly...
# Connect to Redis
r = redis.Redis(host="localhost", port=redis_port)

# Define parking spots data
parking_spots = [
    {"parking_lot_id": 1, "parking_spot_id": 1, "is_free": 1},
    {"parking_lot_id": 1, "parking_spot_id": 2, "is_free": 0},
    {"parking_lot_id": 2, "parking_spot_id": 1, "is_free": 1},
    {"parking_lot_id": 2, "parking_spot_id": 2, "is_free": 1},
]

# Add parking spots to Redis
for spot in parking_spots:
    key = f"parking_spot:{spot['parking_lot_id']}:{spot['parking_spot_id']}"
    r.hset(key, mapping=spot)

# Test if data was added
for spot in parking_spots:
    key = f"parking_spot:{spot['parking_lot_id']}:{spot['parking_spot_id']}"
    print(r.hgetall(key))

{b'parking_lot_id': b'1', b'parking_spot_id': b'1', b'is_free': b'1'}
{b'parking_lot_id': b'1', b'parking_spot_id': b'2', b'is_free': b'0'}
{b'parking_lot_id': b'2', b'parking_spot_id': b'1', b'is_free': b'1'}
{b'parking_lot_id': b'2', b'parking_spot_id': b'2', b'is_free': b'1'}
